In [2]:
import torch
import datasets
import transformers
import numpy as np
from seqeval.metrics import f1_score
import pandas as pd
import os
from torch.utils.data import DataLoader

In [3]:
from train_datasets import BPEDropoutTrainDataset

In [4]:
if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is not enabled.


In [5]:
cache_dir = "./cache"

# Load Data

In [8]:
dataset_path = "msra_ner"
language = None

# subword regularization params:
bpe_dropout_p = 0.0
model_path = "Davlan/xlm-roberta-base-ner-hrl"

In [9]:
train_dataset = BPEDropoutTrainDataset(dataset_path, model_path, dataset_language=language, bpe_dropout_p=bpe_dropout_p, cache_dir=cache_dir, train=True)
test_dataset = BPEDropoutTrainDataset(dataset_path, model_path, dataset_language=language, bpe_dropout_p=0.0, cache_dir=cache_dir, train=False)


Found cached dataset msra_ner (/Users/timwu0/Documents/CS224U/cs224u_subwordreg/cache/msra_ner/msra_ner/1.0.0/5ce47bc7f8da59fd9d0ad08d185fa72f5576b614f136a56e82c7669d22ea5cfe)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset msra_ner (/Users/timwu0/Documents/CS224U/cs224u_subwordreg/cache/msra_ner/msra_ner/1.0.0/5ce47bc7f8da59fd9d0ad08d185fa72f5576b614f136a56e82c7669d22ea5cfe)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
tags = train_dataset.dset['train'].features["ner_tags"].feature
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [33]:
train_dataset.dset['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)

In [31]:
tokenizer = train_dataset.tokenizer

In [32]:
pd.DataFrame(
    [tokenizer.convert_ids_to_tokens(train_dataset[0]['input_ids']), train_dataset[0]['labels']],
    index=["tokens", "ner_tags"])

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
tokens,<s>,▁当,▁,希,▁,望,▁,工,▁,程,...,▁你,▁,悔,▁,不,▁当,▁,初,▁!,</s>
ner_tags,-100,0,0,-100,0,-100,0,-100,0,-100,...,0,0,-100,0,-100,0,0,-100,0,-100


# Training

In [14]:
# Make debugging easier
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [17]:
training_args = transformers.TrainingArguments(
    output_dir = "./checkpoints/xlm-roberta-ner-en-2",
    log_level = "error",
    num_train_epochs = 50,
    per_device_train_batch_size = 12,
    per_device_eval_batch_size = 12,
    evaluation_strategy = "epoch",
    fp16 = False,
    logging_steps = len(train_dataset),
    push_to_hub = False
)

In [18]:
def metrics_func(eval_arg):
    preds = np.argmax(eval_arg.predictions, axis=2)
    batch_size, seq_len = preds.shape
    y_true, y_pred = [], []
    for b in range(batch_size):
        true_label, pred_label = [], []
        for s in range(seq_len):
            if eval_arg.label_ids[b, s] != -100:  # -100 must be ignored
                true_label.append(index2tag[eval_arg.label_ids[b][s]])
                pred_label.append(index2tag[preds[b][s]])
        y_true.append(true_label)
        y_pred.append(pred_label)
    return {"f1": f1_score(y_true, y_pred)}

In [19]:
data_collator = transformers.DataCollatorForTokenClassification(
    tokenizer,
    return_tensors="pt")

In [21]:
xlmr_config = transformers.AutoConfig.from_pretrained(
    model_path,
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index
)

In [28]:
model = (transformers.RobertaForTokenClassification
         .from_pretrained(model_path, config=xlmr_config, cache_dir=cache_dir, ignore_mismatched_sizes=True)
         .to(device))

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at Davlan/xlm-roberta-base-ner-hrl and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
trainer = transformers.Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [30]:
trainer.train()

/Users/timwu0/opt/anaconda3/envs/swreg/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 